---
# 1. Post-Training
----

---
### 1-1. 라이브러리 설치
----

In [1]:
!pip install torch
!pip install transformers==4.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Fri Dec 16 06:58:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

---
### 1-2. 데이터 다운로드
-----

#### 스마일 스타일 데이터세트 (for 대화 및 스타일 트랜스퍼)

In [3]:
!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about

fatal: destination path 'korean_smile_style_dataset' already exists and is not an empty directory.


In [4]:
# tsv을 csv로 바꾸기, 좀 더 다루기 편하게 하기 위해
import pandas as pd
df = pd.read_csv('./korean_smile_style_dataset/smilestyle_dataset.tsv', sep = '\t')
df.to_csv('./korean_smile_style_dataset/smile.csv',index=False)

---
### 1-3. csv 데이터 읽기
---

In [5]:
# 데이터 출력
import csv
data_path = './korean_smile_style_dataset/smile.csv'
f = open(data_path, 'r')
rdr = csv.reader(f)

for line in rdr:
    print(line)
    break

['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']


---
### 1-4. 세션으로 데이터 분할하기
----

In [6]:
f = open(data_path, 'r')
rdr = csv.reader(f)


# 세션 데이터 저장할 것
session_dataset = []
session = []

# 데이터 저장 
for i, line in enumerate(rdr):
    if i == 0:
        header  = line
    else:
        utt = line[0] # formal
        if utt.strip() != '': # 빈 공백이 아니라면
            session.append(utt) # 세션에 추가
        else:  # 빈 공백일 경우 발화의 끝을 의미하니
               # 세션 데이터셋에 저장
            session_dataset.append(session)
            session = []
# 마지막 세션은 따로 저장
session_dataset.append(session)
f.close()

In [7]:
session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [8]:
session_dataset[1]

['나이가 어떻게 되시는데요?',
 '올해로 열일곱 입니다.',
 '그럼 고등학교 1학년이겠네요.',
 '네 맞아요.',
 '장래희망이 어떻게 되세요?',
 '저는 외교관 되는게 꿈이에요.']

---
### 1-5. 토크나이저 확인
----

In [9]:
# https://github.com/sooftware/Korean-PLM
# https://huggingface.co/klue/roberta-base
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [10]:
# 토큰나이저 종류
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_bos_token',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_save_pretrained',
 '_sep_token',
 '_tokenizer',
 '_unk_token',
 'add_special_tokens',
 'add_tokens',
 'additional_special_

In [11]:
# 스페셜 토큰 확인
tokenizer.special_tokens_map_extended

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

- 현재 SEP Token과 EOS Token이 같은 명으로 설정이 되어 있음

      논문입력형태: [CLS] u1 [EOU] ... ut-1 [EOU] [SEP] ut [SEP]
      spe_token을 다른 토큰으로 변경필요
      --> 입력형태: [CLS] u1 <SEP> ... ut-1 <SEP> [SEP] ut [SEP]


In [12]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '[SEP]', 2, 2)

In [13]:
# SEP TOken 변경
special_tokens = {'sep_token': '<SEP>'}
# Tokenizer 변경 후 추가
tokenizer.add_special_tokens(special_tokens)

# 모델도 동일하게 변경
# model.resize_token_embeddings(len(tokenizer))

1

In [14]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '<SEP>', 2, 32000)

---
##### 1-5-1. MLM Masking 예시
----

In [15]:
import random

session = session_dataset[0] # 예시로 하나만 사용
mask_ratio = 0.15 # 마스킹 비율
corrupt_tokens = []
output_tokens = []

for i, utt in enumerate(session):
    original_token = tokenizer.encode(utt, add_special_tokens=False) # Tokenize를 통한 Encoding
    mask_num = int(len(original_token)*mask_ratio) # 마스킹 처리 비율(수) 계산
    mask_positions = random.sample([x for x in range(len(original_token))], mask_num)  # 전체 토큰의 개수 중 마스킹 처리 비율만큼 랜덤으로 추출해서 Masking할 위치를 선정
    corrupt_token = [] # 노이즈 토큰 즉, 마스크된 토큰을 넣는 리스트
    
    for pos in range(len(original_token)):
        if pos in mask_positions: # 현재 마스크된 토큰의 위치에 있다면 
            corrupt_token.append(tokenizer.mask_token_id) # 마스크 토큰 id를 넣어준다
        else: # 아니라면
            corrupt_token.append(original_token[pos]) # 기존의 토큰을 넣어준다.
            
    if i == len(session)-1:
        output_tokens += original_token
        corrupt_tokens += corrupt_token
    else:
        output_tokens += original_token + [tokenizer.sep_token_id] # 문장 사이 구분을 위한 <SEP> Token 추가
        corrupt_tokens += corrupt_token + [tokenizer.sep_token_id]        

In [16]:
print(tokenizer.decode(output_tokens))
print('####')
print(tokenizer.decode(corrupt_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다. 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.
####
[MASK]하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우 [MASK] [MASK] 안 힘드세요? <SEP> [MASK]가 워낙 고양이를 좋아해서 크게 힘들 [MASK] 않아요. <SEP> 가장 나이가 많은 [MASK]가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에 [MASK]. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요 [MASK] <SEP> 한 살입니다. 작년에 분양 [MASK]았어요. <SEP> [MASK] 고양이들끼리 안 싸우나요? <SEP> [MASK] 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요 [MASK]


---
### 1-6. URC을 위한 입력
---

- Utterance Relevance Classfication 
1. 앞에 있는 Context를 잘라서 일부분은 Context, 일부분은 Response로 가정
2. 기존에 있던 Response = Positive Response  
  Random으로 추출한 것 = Negative Response  
  앞에있는 Context에서 추출한 것 = Context Negative Response로 3개의 Class를 구성
3. 해당 데이터를 Classfication Task로 학습을 하는 과정


---
##### 1-6-1. short session 구성 중 Positive Response 만드는 예시
----

In [17]:
# 길이는 임의로 설정
k = 4 # 논문 숫자 참고 (3개는 context, 1개는 response)
short_session_dataset = []
for session in session_dataset:    
    for i in range(len(session)-k+1):
        short_session_dataset.append(session[i:i+k]) # 슬라이딩
print('전체 데이터 Session의 수:',len(short_session_dataset))

전체 데이터 Session의 수: 2762


In [18]:
# 상위 3개는 Context로 사용 
# 마지막 1개는 Positive Response로 사용
short_session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?']

---
##### 1-6-2. short session 구성 중 Negative Response 만드는 예시
----

In [19]:
# 모든 발화가 negative response candidate
import random

all_utts = set() # 모든 발화 저장을 위한 Set 설정

for session in session_dataset: 
    for utt in session:
        all_utts.add(utt) # 모든 발화를 넣기
all_utts = list(all_utts) # 리스트로 변환
print('모든 발화의 개수:',len(all_utts))
print('예시:', all_utts[0])

모든 발화의 개수: 3430
예시: 다음에는 타이타닉 옷을 만드려고 하는데, 고양이가 입어줄까요?


---
##### 1-6-3. URC 예시
----

In [20]:
session = short_session_dataset[0] # 하나 선택 
urc_tokens = []
context_utts = []
for i in range(len(session)):
    utt = session[i]    
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    if i == len(session)-1: # 마지막 발화인 경우 

        ######## 기존 response 입력 -> Positive Response ########
        positive_tokens = urc_tokens + original_token

        ######## random negative respons 입력 ########
        while True:
            random_neg_response = random.choice(all_utts) # 랜덤으로 하나 선택
            if random_neg_response not in context_utts:
                break
        random_neg_response_token = tokenizer.encode(random_neg_response, add_special_tokens=False)
        random_tokens = urc_tokens + random_neg_response_token
        
        ######## context negative response 입력 ########
        context_neg_response = random.choice(context_utts)
        context_neg_response_token = tokenizer.encode(context_neg_response, add_special_tokens=False)
        context_neg_tokens = urc_tokens + context_neg_response_token

    else: # 마지막 발화가 아닌 경우 
        urc_tokens += original_token + [tokenizer.sep_token_id] # SEP Token으로 구분

    context_utts.append(utt) # 모든 발화들을 Context_utt에 추가

In [21]:
print('0번째 Class읠 문장')
print('Positive Class:','\n',tokenizer.decode(positive_tokens))
print(' ')
print('1번째 Class읠 문장')
print('Random Negative Class:','\n',tokenizer.decode(random_tokens)) # 앞의 3개는 동일 마지막만 random uttrance
print(' ')
print('2번째 Class읠 문장')
print('context negative Class:','\n',tokenizer.decode(context_neg_tokens)) # 앞의 3개중에 하나를 뽑는 것

0번째 Class읠 문장
Positive Class: 
 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
 
1번째 Class읠 문장
Random Negative Class: 
 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 일시적인 현상이였으면 좋겠네요.
 
2번째 Class읠 문장
context negative Class: 
 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 안녕하세요. 저는 고양이 6마리 키워요.


---
### 1-7. Post-training 데이터로더 만들기
----

In [22]:
import csv
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class post_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base') # 토큰나이저 불러오기
        special_tokens = {'sep_token': '<SEP>'} # 스페셜 토큰 변경
        self.tokenizer.add_special_tokens(special_tokens)
        
        ################### CSV로 저장된 데이터를 불러서 Session 데이터로 저장 ###################
        f = open(data_path, 'r')
        rdr = csv.reader(f)        
        
        """ 세션 데이터 저장할 것"""
        session_dataset = []
        session = []

        """ 실제 데이터 저장 방식 """
        for i, line in enumerate(rdr):
            if i == 0:
                header  = line
            else:
                utt = line[0]
                if utt.strip() != '':
                    session.append(utt)
                else:
                    """ 세션 데이터 저장 """
                    session_dataset.append(session)
                    session = []
        """ 마지막 세션 저장 """
        session_dataset.append(session)
        f.close()
        #################################################################################
        
        ############# short session context를 만드는 Code #############
        k = 4 # 논문에서 가장 좋았던 숫자
        self.short_session_dataset = []
        for session in session_dataset:    
            for i in range(len(session)-k+1):
                self.short_session_dataset.append(session[i:i+k])
        #############################################################        
        
        ############# Random Negative Response를 이용하기 위해 모든 발화 저장 #############
        self.all_utts = set()
        for session in session_dataset:
            for utt in session:
                self.all_utts.add(utt)
        self.all_utts = list(self.all_utts)
        ############################################################################

    def __len__(self): # 기본적인 구성
        return len(self.short_session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.short_session_dataset[idx]
        
        ########################### MLM 입력 ###########################
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []
        for i, utt in enumerate(session):
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token) * mask_ratio)
            mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(self.tokenizer.mask_token_id) # 마스크된 토큰을 저장
                else:
                    corrupt_token.append(original_token[pos]) # 일반적인 토큰을 저장

            if i == len(session)-1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [self.tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [self.tokenizer.sep_token_id]    
        ################################################################

        ################################ label for loss ###############################
        # 마스크된 부분의 Loss만 계산하기 위한 코드 
        # 마스크가 되지 않은 부분은 자기 자신을 복원하는 것이기 때문에 너무 쉬워 Loss값에 영향을 줄 수 있음
        
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos) # 마스크된 포지션의 정보를 저장               
        ###############################################################################                
        
        ###################################### URC 입력 ######################################
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]    
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)
            if i == len(session)-1:
                urc_tokens += [self.tokenizer.eos_token_id] # Response 해당하는 부분을 구별하기 위핸 EOS 토큰 삽입
                """ 기존 response 입력 """
                self.positive_tokens = [self.tokenizer.cls_token_id] + urc_tokens + original_token # cls token 추가

                """ random negative respons 입력 """
                while True:
                    random_neg_response = random.choice(self.all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(random_neg_response, add_special_tokens=False)
                self.random_tokens = [self.tokenizer.cls_token_id] + urc_tokens + random_neg_response_token
                """ context negative response 입력 """
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(context_neg_response, add_special_tokens=False)
                self.context_neg_tokens = [self.tokenizer.cls_token_id] + urc_tokens + context_neg_response_token
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)
        #########################################################################################

        return self.corrupt_tokens, self.output_tokens, self.corrupt_mask_positions, [self.positive_tokens, self.random_tokens, self.context_neg_tokens], [0, 1, 2]

In [23]:
data_path = './korean_smile_style_dataset/smile.csv'
post_dataset = post_loader(data_path)

In [24]:
corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = post_dataset[0]

In [25]:
print('\n','마스킹된 문장')
print(post_dataset.tokenizer.decode(corrupt_tokens))
print('\n','원본 문장')
print(post_dataset.tokenizer.decode(output_tokens))
print('\n','마스크의 위치')
print(corrupt_mask_positions)


 마스킹된 문장
안녕하세요. 저는 [MASK] 6마리 키워요. <SEP> 고양이를 6마리나 [MASK] [MASK] 키우는거 안 힘드세요? <SEP> [MASK]가 워낙 고양이 [MASK] 좋아해서 크게 힘들진 않아요. <SEP> 가장 [MASK]가 많은 고양이가 어떻게 돼요?

 원본 문장
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?

 마스크의 위치
[6, 18, 19, 28, 32, 44]


In [26]:
print('\n','Positive Class 문장:')
print(post_dataset.tokenizer.decode(urc_inputs[0]))
print('\n','Random Negative Class 문장:')
print(post_dataset.tokenizer.decode(urc_inputs[1]))
print('\n','context negative Class 문장:')
print(post_dataset.tokenizer.decode(urc_inputs[2]))
print('\n', '라벨값')
print(urc_labels)


 Positive Class 문장:
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 가장 나이가 많은 고양이가 어떻게 돼요?

 Random Negative Class 문장:
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 큰 가시였나요? 빼기 힘들었을텐데요.

 context negative Class 문장:
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 고양이를 6마리나요? 키우는거 안 힘드세요?

 라벨값
[0, 1, 2]


---
### 1-8. 배치 처리부분 추가
----

- URC 부분은 배치가 3의 배수로 설정이 됨


            input:
                data: [(session1), (session2), ... ]
            return:
                batch_corrupt_tokens: (B, L) padded
                batch_output_tokens: (B, L) padded
                batch_corrupt_mask_positions: list
                batch_urc_inputs: (B, L) padded
                batch_urc_labels: (B)
                batch_mlm_attentions
                batch_urc_attentions

            batch가 3
            MLM = 3개의 입력데이터 (입력데이터별로 길이가 다름)
            URC = 9개의 입력데이터 (context는 길이가 다름, response candidate도 길이가 다름)


In [27]:
import csv
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class post_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base') # 토큰나이저 불러오기
        special_tokens = {'sep_token': '<SEP>'} # 스페셜 토큰 변경
        self.tokenizer.add_special_tokens(special_tokens)
        
        ################### CSV로 저장된 데이터를 불러서 Session 데이터로 저장 ###################
        f = open(data_path, 'r')
        rdr = csv.reader(f)        
        
        """ 세션 데이터 저장할 것"""
        session_dataset = []
        session = []

        """ 실제 데이터 저장 방식 """
        for i, line in enumerate(rdr):
            if i == 0:
                header  = line
            else:
                utt = line[0]
                if utt.strip() != '':
                    session.append(utt)
                else:
                    """ 세션 데이터 저장 """
                    session_dataset.append(session)
                    session = []
        """ 마지막 세션 저장 """
        session_dataset.append(session)
        f.close()
        #################################################################################
        
        ############# short session context를 만드는 Code #############
        k = 4 # 논문에서 가장 좋았던 숫자
        self.short_session_dataset = []
        for session in session_dataset:    
            for i in range(len(session)-k+1):
                self.short_session_dataset.append(session[i:i+k])
        #############################################################        
        
        ############# Random Negative Response를 이용하기 위해 모든 발화 저장 #############
        self.all_utts = set()
        for session in session_dataset:
            for utt in session:
                self.all_utts.add(utt)
        self.all_utts = list(self.all_utts)
        ############################################################################

    def __len__(self): # 기본적인 구성
        return len(self.short_session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.short_session_dataset[idx]
        
        ########################### MLM 입력 ###########################
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []
        for i, utt in enumerate(session):
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token)*mask_ratio)
            mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(self.tokenizer.mask_token_id) # 마스크된 토큰을 저장
                else:
                    corrupt_token.append(original_token[pos]) # 일반적인 토큰을 저장

            if i == len(session)-1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [self.tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [self.tokenizer.sep_token_id]    
        ################################################################

        ################################ label for loss ###############################
        # 마스크된 부분의 Loss만 계산하기 위한 코드 
        # 마스크가 되지 않은 부분은 자기 자신을 복원하는 것이기 때문에 너무 쉬워 Loss값에 영향을 줄 수 있음
        
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos) # 마스크된 포지션의 정보를 저장               
        ###############################################################################                
        
        ###################################### URC 입력 ######################################
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]    
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)
            if i == len(session)-1:
                urc_tokens += [self.tokenizer.eos_token_id] # Response 해당하는 부분을 구별하기 위핸 EOS 토큰 삽입
                """ 기존 response 입력 """
                self.positive_tokens = [self.tokenizer.cls_token_id] + urc_tokens + original_token # cls token 추가

                """ random negative respons 입력 """
                while True:
                    random_neg_response = random.choice(self.all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(random_neg_response, add_special_tokens=False)
                self.random_tokens = [self.tokenizer.cls_token_id] + urc_tokens + random_neg_response_token
                """ context negative response 입력 """
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(context_neg_response, add_special_tokens=False)
                self.context_neg_tokens = [self.tokenizer.cls_token_id] + urc_tokens + context_neg_response_token
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)
        #########################################################################################
        
        return self.corrupt_tokens, self.output_tokens, self.corrupt_mask_positions, [self.positive_tokens, self.random_tokens, self.context_neg_tokens], [0, 1, 2]

    def collate_fn(self, sessions): # 배치를 위한 구성

        batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels = [], [], [], [], []
        batch_mlm_attentions, batch_urc_attentions = [], []

        ###################################### MLM, URC 입력에 대해서 가장 긴 입력 길이를 찾기 ######################################
        corrupt_max_len, urc_max_len = 0, 0
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            if len(corrupt_tokens) > corrupt_max_len: 
                corrupt_max_len = len(corrupt_tokens) # 길이가 더 긴 토큰으로 갱신

            positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            if max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)]) > urc_max_len:
                urc_max_len = max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)]) # 길이가 더 긴 토큰으로 갱신
        #####################################################################################################################


        #################################################### padding 토큰을 추가하는 부분 ####################################################
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            """ mlm 입력 """ # 마스크된 문장 토큰 부분
            batch_corrupt_tokens.append(corrupt_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len-len(corrupt_tokens))])
            batch_mlm_attentions.append([1 for _ in range(len(corrupt_tokens))] + [0 for _ in range(corrupt_max_len-len(corrupt_tokens))])
            
            """ mlm 출력 """ # 복원시킬 토큰 부분 즉, 원본
            batch_output_tokens.append(output_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len-len(corrupt_tokens))])
            
            """ mlm 레이블 """ # 마스크된 부분의 Loss 처리를 위해
            batch_corrupt_mask_positions.append(corrupt_mask_positions)
            
            """ urc 입력 """
            # positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            for urc_input in urc_inputs:                            
                batch_urc_inputs.append(urc_input + [self.tokenizer.pad_token_id for _ in range(urc_max_len-len(urc_input))]) # 현 토큰의 길이와 가장 긴 토큰의 길이를 뺀 나머지를 패딩 처리
                batch_urc_attentions.append([1 for _ in range(len(urc_input))] + [0 for _ in range(urc_max_len-len(urc_input))]) # padding token Attention 비처리를 위한 코드
            
            """ urc 레이블 """
            batch_urc_labels += urc_labels
        ###################################################################################################################################

        return torch.tensor(batch_corrupt_tokens), torch.tensor(batch_output_tokens), batch_corrupt_mask_positions, torch.tensor(batch_urc_inputs), \
        torch.tensor(batch_urc_labels), torch.tensor(batch_mlm_attentions), torch.tensor(batch_urc_attentions)    

In [28]:
from torch.utils.data import DataLoader
data_path = './korean_smile_style_dataset/smile.csv'
post_dataset = post_loader(data_path)
post_dataloader = DataLoader(post_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=post_dataset.collate_fn)

In [29]:
for data in post_dataloader:
    batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels, batch_mlm_attentions, batch_urc_attentions = data
    break

In [30]:
print(batch_corrupt_tokens.shape) # (배치, 가장 긴 입력 데이터)
print(batch_output_tokens.shape) # (배치, 가장 긴 입력 데이터)
print(batch_corrupt_mask_positions) # 마스크된 부분의 Position
print(batch_urc_inputs.shape) # (3 x 배치, 가장 긴 입력 데이터)
print(batch_urc_labels.shape) # (3 x 배치, 가장 긴 입력 데이터)

torch.Size([2, 55])
torch.Size([2, 55])
[[1, 19, 21, 39, 51], [0, 15, 23, 25, 40]]
torch.Size([6, 65])
torch.Size([6])


In [31]:
post_dataset.tokenizer.decode(batch_corrupt_tokens[0,:]), post_dataset.tokenizer.decode(batch_output_tokens[0,:])

('어떤 [MASK]에서는 철학적으로 들리네요. <SEP> 친구들이랑 같이 낚시하러 가면 [MASK]를 [MASK] 기회도 생겨서 좋습니다. <SEP> 주로 친구들이랑 낚시하러 가시나 [MASK]? <SEP> 아니요, 혼자 낚시하는 날이 가장 [MASK]습니다.',
 '어떤 면에서는 철학적으로 들리네요. <SEP> 친구들이랑 같이 낚시하러 가면 대화를 할 기회도 생겨서 좋습니다. <SEP> 주로 친구들이랑 낚시하러 가시나요? <SEP> 아니요, 혼자 낚시하는 날이 가장 많습니다.')

In [32]:
batch_mlm_attentions

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

In [33]:
post_dataset.tokenizer.decode(batch_urc_inputs[0]), batch_urc_labels

('[CLS] 어떤 면에서는 철학적으로 들리네요. <SEP> 친구들이랑 같이 낚시하러 가면 대화를 할 기회도 생겨서 좋습니다. <SEP> 주로 친구들이랑 낚시하러 가시나요? <SEP> [SEP] 아니요, 혼자 낚시하는 날이 가장 많습니다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 tensor([0, 1, 2, 0, 1, 2]))

---
### 1-9. Modeling
---

---
##### 1-9-1. 어떤 MLM model을 써야 하나?
----

In [34]:
from transformers import AutoModel, AutoTokenizer, RobertaForMaskedLM, RobertaModel

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
special_tokens = {'sep_token': '<SEP>'}
tokenizer.add_special_tokens(special_tokens)

model1 = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
model2 = RobertaModel.from_pretrained('klue/roberta-base') # AutoModel대신 roberta 방식
model3 = AutoModel.from_pretrained('klue/roberta-base') # klue 공식 document에 있는 방식

# 변경된 토큰나이저를 모델에도 동일하게 변경
model1.resize_token_embeddings(len(tokenizer)) 
model2.resize_token_embeddings(len(tokenizer))
model3.resize_token_embeddings(len(tokenizer))

# Inference 단계에서 모델의 출력이 달라지는 것을 방지하기 위해
model1.eval()
model2.eval()
model3.eval()
print('')

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# output_hidden_states => 여러개의 hidden_state를 하나씩 보기 위해 True로 설정
result = model1(batch_corrupt_tokens, output_hidden_states=True)
print(result.keys())

# 마지막 Layer를 출력
result['hidden_states'][-1], result['hidden_states'][-1].shape

odict_keys(['logits', 'hidden_states'])


(tensor([[[-0.0472, -0.6018, -0.1360,  ...,  0.0557, -0.0038,  0.0956],
          [ 0.3178, -0.1203,  0.2846,  ...,  0.0269, -0.0214,  0.1763],
          [ 0.4524,  0.3233,  0.7471,  ...,  0.3661, -0.0936, -0.5332],
          ...,
          [ 0.2693, -0.0547,  0.4443,  ..., -0.0858, -0.0039,  0.1360],
          [-0.3151, -0.2606, -0.1496,  ..., -0.3539, -0.0044, -0.0333],
          [-0.4269, -0.3260, -0.4910,  ..., -0.1925,  0.0767, -0.0547]],
 
         [[-0.1893, -0.4950, -0.4411,  ..., -0.1591, -0.1413,  0.2338],
          [-0.0782,  0.1383,  0.1443,  ..., -0.1518, -0.0213,  0.4242],
          [ 0.1979, -0.1832,  0.5900,  ..., -0.0111, -0.7170, -0.4477],
          ...,
          [-0.6828, -0.0704, -1.0441,  ...,  0.1045, -0.4524,  0.2687],
          [-0.6828, -0.0704, -1.0441,  ...,  0.1045, -0.4524,  0.2687],
          [-0.6828, -0.0704, -1.0441,  ...,  0.1045, -0.4524,  0.2687]]],
        grad_fn=<NativeLayerNormBackward0>), torch.Size([2, 55, 768]))

In [36]:
# 마지막 hidden_state에 Matrix를 곱한 값 즉, 최종 출력
result['logits'].shape
# MLM이라는 것은 masking 토큰을 reconstruction하는 것이기 떄문에
# 출력의 dimension이 vocab size여야 된다.
# 그러기 떄문에 출력 dimension = 32001이어야 된다.

torch.Size([2, 55, 32001])

---
##### 1-9-2. RobertaModel과 AutoModel은 같은가?
---

- model2와 model3에서 출력되는 값은 동일해야 함

- 그러나 값이 조금 다르다
- 해당 이유는 resize_token_embedding을 하는 과정에서 모델의 vocab_szie가 달라진다.
- 새로 생긴 vocab에 대한 weight의 값이 랜덤 초기화가 되기 때문에 
- 랜덤 초기화된 Matrix로 인해 Attenion의 값이 달라지게 되기 때문
- 이론 상으로는 동일해야 함

In [37]:
result2 = model2(batch_corrupt_tokens, output_hidden_states=True)
print(result2.keys(),'\n')

# last_hidden_state = 마지막 hidden_state
# hidden_states = 각각의 hidden_state Layer의 출력을 모아둔 것
# pooler_output = last_hidden_state를 pooler layer에 입력해 얻은 값 / 최종 출력값의 Vector Sequence
print('last_hidden_state shape:','\n',result2['last_hidden_state'].shape)
print('pooler_output shape:', '\n',result2['pooler_output'].shape)
print(' ')
print(result2['hidden_states'][-1])

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states']) 

last_hidden_state shape: 
 torch.Size([2, 55, 768])
pooler_output shape: 
 torch.Size([2, 768])
 
tensor([[[-0.0280, -0.5389, -0.0785,  ...,  0.0514, -0.0480,  0.1722],
         [ 0.4427, -0.0804,  0.3899,  ...,  0.0748, -0.1232,  0.0904],
         [ 0.5117,  0.2253,  0.8582,  ...,  0.3978, -0.1460, -0.4934],
         ...,
         [ 0.2050, -0.0433,  0.3874,  ...,  0.0369,  0.0064,  0.2508],
         [-0.3186, -0.1455, -0.1530,  ..., -0.1014, -0.0745,  0.0555],
         [-0.4010, -0.0625, -0.5303,  ..., -0.1015,  0.1201, -0.1451]],

        [[-0.1781, -0.4295, -0.2996,  ..., -0.1636, -0.2112,  0.2828],
         [-0.0389,  0.2431,  0.4980,  ..., -0.2133, -0.1624,  0.4324],
         [ 0.2779, -0.1829,  0.6524,  ..., -0.0430, -0.7799, -0.3337],
         ...,
         [-0.7122,  0.0271, -0.9234,  ...,  0.1195, -0.5021,  0.2540],
         [-0.7122,  0.0271, -0.9234,  ...,  0.1195, -0.5021,  0.2540],
         [-0.7122,  0

In [38]:
result3 = model3(batch_corrupt_tokens, output_hidden_states=True)
print(result3.keys(),'\n')
print('last_hidden_state shape:','\n',result3['last_hidden_state'].shape)
print('pooler_output shape:', '\n',result3['pooler_output'].shape)
print(' ')
print(result3['hidden_states'][-1])

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states']) 

last_hidden_state shape: 
 torch.Size([2, 55, 768])
pooler_output shape: 
 torch.Size([2, 768])
 
tensor([[[-0.0208, -0.5350, -0.0413,  ...,  0.0828, -0.0056,  0.1397],
         [ 0.4385,  0.0238,  0.4684,  ...,  0.1316, -0.2845, -0.0585],
         [ 0.5239,  0.2802,  0.8257,  ...,  0.4425, -0.1955, -0.4904],
         ...,
         [ 0.2258, -0.0225,  0.4217,  ..., -0.0503, -0.0080,  0.2309],
         [-0.3852, -0.1468, -0.0976,  ..., -0.1394, -0.0771,  0.0398],
         [-0.3336, -0.0930, -0.3124,  ..., -0.1090,  0.1435, -0.0959]],

        [[-0.2095, -0.4437, -0.3448,  ..., -0.1448, -0.1804,  0.1761],
         [-0.1441,  0.2827,  0.2301,  ..., -0.0374, -0.1059,  0.2959],
         [ 0.2153, -0.1302,  0.7137,  ...,  0.0220, -0.8117, -0.3185],
         ...,
         [-0.7685, -0.0016, -0.8544,  ...,  0.1439, -0.4981,  0.1996],
         [-0.7685, -0.0016, -0.8544,  ...,  0.1439, -0.4981,  0.1996],
         [-0.7685, -0

---
##### 1-9-3. RobertaForMaskedLM을 사용한 MLM을 위한 Modeling
----

In [39]:
## MLM: masking 토큰을 본 토큰으로 재구성하기: 출력이 vocab_size가 되어야함. RobertaForMaskedLM을 사용하면 됨
## URC: response candidate 3개의 클래스로 분류하는 작업

In [40]:
from transformers import RobertaForMaskedLM, AutoTokenizer
import torch
import torch.nn as nn
# import pdb

########## MLM을 위한 Modeling ##########
class PostModel(nn.Module):
    def __init__(self):
        super(PostModel, self).__init__()
        self.model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
        self.hiddenDim = self.model.config.hidden_size
        
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        self.model.resize_token_embeddings(len(self.tokenizer))        
        
        # Class에 해당하는 Matrix 설정
        # URC를 하기위해 출력의 Logits이 3 필요하기 때문
        """ score matrix """ 
        self.W = nn.Linear(self.hiddenDim, 3)
        
    def forward(self, batch_corrupt_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_mlm_attentions, batch_urc_attentions):        
        
        ############## MLM ##############
        corrupt_outputs = self.model(batch_corrupt_tokens, attention_mask=batch_mlm_attentions)['logits'] # [batch, max_len, 32001]

        corrupt_mask_outputs = []

        # loss를 계산할 부분은 mask된 부분
        # 해당 값을 알기 위해서는 mask position을 가져와 Tokem의 출력을 확인
        for i_batch in range(len(batch_corrupt_mask_positions)): 
            corrupt_mask_output = []
            batch_corrupt_mask_position = batch_corrupt_mask_positions[i_batch]
            for pos in batch_corrupt_mask_position:
                corrupt_mask_output.append(corrupt_outputs[i_batch, pos, :].unsqueeze(0)) # [1, 1, 32001] -> [1, 32001]
            # mask된 position의 출력들을 concat
            corrupt_mask_outputs.append(torch.cat(corrupt_mask_output, 0)) # [mask_num, 32001]
        
        ############## URC ##############
        urc_outputs = self.model(batch_urc_inputs, attention_mask=batch_urc_attentions, output_hidden_states=True)['hidden_states'][-1] # [3 x batch, max_len, 768]
        urc_logits = self.W(urc_outputs) # [3 x batch, max_len, Class의 개수]
        
        # [:,0,:]을 통해 CLS Token의 Vector를 출력
        urc_cls_outputs = urc_logits[:,0,:] # [3 x batch, Class의 개수]
        
        # pdb.set_trace() # 디버깅 코드
        return corrupt_mask_outputs, urc_cls_outputs

In [42]:
post_model = PostModel()
corrupt_mask_outputs, urc_cls_outputs = post_model(batch_corrupt_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_mlm_attentions, batch_urc_attentions)

In [43]:
corrupt_mask_outputs[0].shape, corrupt_mask_outputs[1].shape

(torch.Size([5, 32001]), torch.Size([5, 32001]))

In [44]:
urc_cls_outputs.shape

torch.Size([6, 3])

---
### 1-10. Loss
---

In [45]:
import torch.nn as nn

# pred_outs: [batch, cls_Num]
# labels: [batch]

def CELoss(pred_outs, labels):

    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs, labels)
    return loss_val

---
### 1-11. Train
----

In [46]:
# 모델 저장

import os
def SaveModel(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, 'post_model.bin'))

In [47]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import pdb

post_model = PostModel().cuda()

training_epochs = 5 
max_grad_norm = 10 # gradient explording 방지용
lr = 1e-5
num_training_steps = len(post_dataset)*training_epochs 
num_warmup_steps = len(post_dataset)
optimizer = torch.optim.AdamW(post_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # 학습률 변경하면서 학습을 위해

In [49]:
for epoch in range(training_epochs):
    print(f"{epoch+1}번째 학습시작!!")
    post_model.train() 
    for i_batch, data in enumerate(tqdm(post_dataloader)):
        batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels, batch_mlm_attentions, batch_urc_attentions = data
        
        ######## GPU 할당 ########
        batch_corrupt_tokens = batch_corrupt_tokens.cuda()
        batch_output_tokens = batch_output_tokens.cuda()
        batch_urc_inputs = batch_urc_inputs.cuda()
        batch_urc_labels = batch_urc_labels.cuda()
        batch_mlm_attentions = batch_mlm_attentions.cuda()
        batch_urc_attentions = batch_urc_attentions.cuda()
      
        
        ########## Prediction ##########
        corrupt_mask_outputs, urc_cls_outputs = post_model(batch_corrupt_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_mlm_attentions, batch_urc_attentions)

        ########## Loss calculation & training ##########
        original_token_indexs = [] # mask된 Token에 대한 원본 Token의 index
        for i_batch in range(len(batch_corrupt_mask_positions)):
            original_token_index = []
            batch_corrupt_mask_position = batch_corrupt_mask_positions[i_batch]
            for pos in batch_corrupt_mask_position:
                original_token_index.append(batch_output_tokens[i_batch,pos].item())
            original_token_indexs.append(original_token_index)
        
        ###### MLM Loss ######
        mlm_loss = 0
        for corrupt_mask_output, original_token_index in zip(corrupt_mask_outputs, original_token_indexs):
              # corrupt_mask_output => masking된 Token의 Predict값 
              # original_token_index => 정답값
            mlm_loss += CELoss(corrupt_mask_output, torch.tensor(original_token_index).cuda()) # Tensor로 변경 후 GPU 동일하게 할당해서 비교       
        
        ###### URC Loss ######
        urc_loss = CELoss(urc_cls_outputs, batch_urc_labels)

        loss_val = mlm_loss + urc_loss
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(post_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()   
        
SaveModel(post_model, '.')

0번째 학습시작!!


  0%|          | 0/1381 [00:00<?, ?it/s]

1번째 학습시작!!


  0%|          | 0/1381 [00:00<?, ?it/s]

2번째 학습시작!!


  0%|          | 0/1381 [00:00<?, ?it/s]

3번째 학습시작!!


  0%|          | 0/1381 [00:00<?, ?it/s]

4번째 학습시작!!


  0%|          | 0/1381 [00:00<?, ?it/s]

---
# 2. Fine-Tuning
----

---
### 2-1. Train % Validation Set 분할
---

In [50]:
import csv
f = open('./korean_smile_style_dataset/smile.csv','r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것"""
session_dataset = []
session = []
all_utts = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header  = line
    else:
        utt = line[0]
        if utt.strip() != '':
            all_utts.append(utt)
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []        
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [51]:
train_session = session_dataset[:-15]
valid_session = session_dataset[-15:]

print('전체 Session의 개수:',len(session_dataset))
print('Train Session의 개수:',len(train_session))
print('Validation Session의 개수:',len(valid_session))

전체 Session의 개수: 236
Train Session의 개수: 221
Validation Session의 개수: 15


---
### 2-2. positive / negative pairs 구성
---

- 데이터의 개수가 부족하기 때문에 Argumentation 개념으로 데이터 슬라이딩 진행

      첫번째 데이터
      context: 안녕하세요. 저는 고양이 6마리 키워요.
      response: 고양이를 6마리나요? 키우는거 안 힘드세요?

      두번째 데이터
      context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요?
      response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.

      세번째 데이터
      context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요? | 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.
      response: 가장 나이가 많은 고양이가 어떻게 돼요?


- Train set

In [52]:
import random

use_turns = 5 # context 개수를 제한하기 위해 ues_turns 사용
neg_nums = 4 # negative의 개수 설정

from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        train_json[count]['context'] = context[:][-use_turns:]
        train_json[count]['positive_response'] = utt
        context.append(utt)        
        
        negative_candidates = random.sample(all_utts, neg_nums)
        train_json[count]['negative_responses'] = negative_candidates
        count += 1


# 데이터셋 저장    
import json
with open('./korean_smile_style_dataset/train.json', 'w') as outfile:
    json.dump(train_json, outfile)

- Validation Set

In [53]:
import random
neg_nums = 4

from collections import defaultdict
dev_json = defaultdict(dict)
count = 0
for session in valid_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        dev_json[count]['context'] = context[:][-use_turns:]
        dev_json[count]['positive_response'] = utt        
        context.append(utt)        
        
        negative_candidates = random.sample(all_utts, neg_nums)
        dev_json[count]['negative_responses'] = negative_candidates
        count += 1
    
import json
with open('./korean_smile_style_dataset/dev.json', 'w') as outfile:
    json.dump(dev_json, outfile)

---
# 3. DataLoader
----

In [54]:
import json, pdb
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class fine_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        # 저장한 데이터셋 불러오기 (정적 샘플링)    / 만일 동적 샘플링을 진행하려면 위 데이터셋 코드를 삽입
        with open(data_path, 'r') as f:
            self.session_dataset = json.load(f)
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.session_dataset[str(idx)]
        context = session['context']
        positive_response = session['positive_response']
        negative_responses = session['negative_responses']
        session_tokens = []
        session_labels = []
        
        """ MRS 입력 """

        # 기존 <SEP> Token 만 사용해도 모델에는 문제가 없지만 
        # [SEP] Token을 사용할 경우 Response 문장과 [CLS] Token과의 Attention에 있어서 더 효율적인 모습을 보여줌
        context_token = [self.tokenizer.cls_token_id]
        for utt in context:
            context_token += self.tokenizer.encode(utt, add_special_tokens=False)
            context_token += [self.tokenizer.sep_token_id] # <SEP> Token
        
        pos_respons_token = [self.tokenizer.eos_token_id] # [SEP] Token 먼저 삽입
        pos_respons_token += self.tokenizer.encode(positive_response, add_special_tokens=False) # Response 응답 삽입
        positive_tokens = context_token + pos_respons_token
        session_tokens.append(positive_tokens)
        session_labels.append(1) # Positive Token Label
        
        for negative_response in negative_responses:
            neg_respons_token = [self.tokenizer.eos_token_id] # [SEP] Token
            neg_respons_token += self.tokenizer.encode(negative_response, add_special_tokens=False)
            negative_tokens = context_token + neg_respons_token        
            session_tokens.append(negative_tokens)
            session_labels.append(0) # Negative Token Label
        
        return session_tokens, session_labels
    
    def collate_fn(self, sessions): # 배치를 위한 구성

        ########### padding을 위한 최대 길이 찾기 ###########
        max_input_len = 0
        for session in sessions:
            session_tokens, session_labels = session
            input_tokens_len = [len(x) for x in session_tokens]
            max_input_len = max(max_input_len, max(input_tokens_len))
        ################################################

        batch_input_tokens, batch_input_labels = [], []
        batch_input_attentions = []
        for session in sessions:
            session_tokens, session_labels = session
            for session_token in session_tokens:
                input_token = session_token + [self.tokenizer.pad_token_id for _ in range(max_input_len-len(session_token))] # Max_input_len 만큼 padding Token 채우기
                input_attention = [1 for _ in range(len(session_token))] + [0 for _ in range(max_input_len-len(session_token))] # padding Token 부분 Attention 미처리
                batch_input_tokens.append(input_token)
                batch_input_attentions.append(input_attention)
            batch_input_labels += session_labels
        
        return torch.tensor(batch_input_tokens), torch.tensor(batch_input_attentions), torch.tensor(batch_input_labels)

In [78]:
train_path = './korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=train_dataset.collate_fn)

In [79]:
dev_path = './korean_smile_style_dataset/dev.json'
dev_dataset = fine_loader(dev_path)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

In [80]:
for data in train_dataloader:
    batch_input_tokens, batch_input_attentions, batch_input_labels = data
    break

In [81]:
batch_input_tokens.shape, batch_input_attentions.shape, batch_input_labels.shape
# 배치 x 5(코드상 처리된 부분)

(torch.Size([5, 36]), torch.Size([5, 36]), torch.Size([5]))

In [82]:
batch_input_labels

tensor([1, 0, 0, 0, 0])

In [83]:
batch_input_tokens

tensor([[    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  7003,  2138,    26, 12736,  2075,
          2182,    35,  5688,  2259,  2180,  1378, 18941,  5971,    35,     1,
             1,     1,     1,     1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2, 15641,  2119,  1560,  2073,  2147,
            16,  5170,  4217,  2170,  1363,  2118,  2259,  5794,  2182,    18,
             1,     1,     1,     1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  5139,  1560,  2073,  5665,   555,
          2203,  2182,    18,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  1535,  22

---
# 3. Modeling
---

In [84]:
class FineModel(nn.Module):
    def __init__(self):
        super(FineModel, self).__init__()
        self.model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
        self.hiddenDim = self.model.config.hidden_size
        
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        self.model.resize_token_embeddings(len(self.tokenizer))        
        
        """ score matrix """
        # Class의 개수는 2개 (positive, negative)
        self.W2 = nn.Linear(self.hiddenDim, 2)
        
    def forward(self, batch_input_tokens, batch_input_attentions):
        ## Binary Classification (pointwise)
        outputs = self.model(batch_input_tokens, attention_mask=batch_input_attentions, output_hidden_states=True)['hidden_states'][-1] # [Batch, Logits, hidden_dim]
        cls_outputs = outputs[:,0,:] # [B, hidden_dim]
        cls_logits = self.W2(cls_outputs) # [B, 2]
         
        return cls_logits

---
### 3-1. Post-Traning model loading
----

In [85]:
fine_model = FineModel().cuda()

# 기존 post-train(post_model.bin)의 경우 마지막 shape은 [B,3]이나 
# 현 fine-tuning의 마지막 shape은 [B,2] 이기 때문에 이를 맞춰 주어야 한다. 
fine_model.load_state_dict(torch.load('post_model.bin'), strict=False)


_IncompatibleKeys(missing_keys=['W2.weight', 'W2.bias'], unexpected_keys=['W.weight', 'W.bias'])

---
# 4. Evaluation Method
---

      label: [1, 0, 0, 1, 0, 0]
      pred:  [1, 0, 1, 0, 1, 0]
      precision = 1/3
      recall = 1/2

      100명의 입원자가있고 암발생을 검진하는 AI. 실제로는 99명은 암에 안걸렸고 1명이 암걸렸습니다.
      모델입장에서 모두 암에 안걸렸다고 하면? accuracy = 99%
      recall = 0/1

      모델입장에서 10명 암에걸렸다고 판단. 이중에는 실제 암환자가 있었다.
      precision = 1/10, recall = 1/1

      10개중 9개가 negative 1개가 positive
      R10@1: response candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/1, 0/1
      R10@5: response candidate가 10개가 있다. 5이라는 것은 top-5을 보겠다. --> 1/1, 0/1

      10개중 7개가 negative 3개가 positve (Douban은 positive 개수가 매번 달라짐)
      R10@1: response candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/3, 0/3
      P@1: response candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/1, 0/1

      결과적으로 이 실습에서는 R@1을 사용

In [86]:
from torch.nn.functional import softmax

def CalP1(fine_model, dataloader):
    fine_model.eval()
    correct = 0
    for i_batch, data in enumerate(tqdm(dataloader, desc="evaluation")):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data
        
        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        
        # batch_input_labels: [1,0,0,0,0]  <== Batch_size 1인 경우 
        batch_input_labels = batch_input_labels.cuda() 
        
        """Prediction"""
        outputs = fine_model(batch_input_tokens, batch_input_attentions)    
        probs = softmax(outputs, 1)
        true_probs = probs[:,1] # positive에 해당 하는 확률
        pred_ind = true_probs.argmax(0).item() # pred_inx : 0
        gt_ind = batch_input_labels.argmax(0).item() # gt_inx : 0
        
        if pred_ind == gt_ind:
            correct += 1
    return round(correct/len(dataloader)*100, 2)

---
# 5. Train
----

In [87]:
import os
def SaveModel_fine(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, 'fine_model.bin'))

In [88]:
len(train_dataloader), len(dev_dataloader)

(3020, 214)

In [89]:
training_epochs = 5
max_grad_norm = 10
lr = 1e-5
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(fine_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [90]:
best_p1 = 0
for epoch in range(training_epochs):
    fine_model.train() 
    for i_batch, data in enumerate(tqdm(train_dataloader)):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data
        
        ##### GPU 할당 #####
        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        batch_input_labels = batch_input_labels.cuda()
        
        ######## Prediction ########
        outputs = fine_model(batch_input_tokens, batch_input_attentions)
        loss_val = CELoss(outputs, batch_input_labels)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(fine_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    ######## 평가 #########
    fine_model.eval()
    p1 = CalP1(fine_model, dev_dataloader)
    print(f"Epoch: {epoch+1}번째 모델 성능(p@1): {p1}")
    if p1 > best_p1:
        best_p1 = p1
        print(f"BEST 성능(p@1): {best_p1}")
        SaveModel_fine(fine_model, '.')

  0%|          | 0/3020 [00:00<?, ?it/s]

evaluation:   0%|          | 0/214 [00:00<?, ?it/s]

Epoch: 1번째 모델 성능(p@1): 95.33
BEST 성능(p@1): 95.33


  0%|          | 0/3020 [00:00<?, ?it/s]

evaluation:   0%|          | 0/214 [00:00<?, ?it/s]

...

evaluation:   0%|          | 0/214 [00:00<?, ?it/s]

Epoch: 4번째 모델 성능(p@1): 94.39


  0%|          | 0/3020 [00:00<?, ?it/s]

evaluation:   0%|          | 0/214 [00:00<?, ?it/s]

Epoch: 5번째 모델 성능(p@1): 95.79


---
# 6. Inference
----

In [73]:
fine_model = FineModel().cuda()
fine_model.load_state_dict(torch.load('fine_model.bin')) # 저장된 모델 불러오기

<All keys matched successfully>

In [74]:
# 모델이 학습되는 데이터가 너무 적어서 다양한 입력 형태를 반영하기 어렵다.
# 컨텍스트는 dev 셋에서 하나의 샘플을 가져옴
context = ["어떤 문제가 있으신가요?", "어떤 차를 사야 할지 잘 모르겠어요.", "차는 한 번 사면 10년도 넘게 써서, 신중하게 골라야 해요."]
candidates = ["저 좀 도와주실 수 있나요?", "저는 농구를 좋아합니다.", "자동차는 신중히 골라야합니다.", "저는 차 마시는거를 좋아합니다.", "날씨가 화창합니다.", "저는 차를 좋아합니다."]

---
### 6-1. Context와 Response Token화 및 Encoding
----

In [75]:
context_token = [fine_model.tokenizer.cls_token_id]
for utt in context:
    context_token += fine_model.tokenizer.encode(utt, add_special_tokens=False)
    context_token += [fine_model.tokenizer.sep_token_id]

session_tokens = []    
for response in candidates:
    response_token = [fine_model.tokenizer.eos_token_id]
    response_token += fine_model.tokenizer.encode(response, add_special_tokens=False)
    candidate_tokens = context_token + response_token        
    session_tokens.append(candidate_tokens)
  

----
### 6-2. Padding 처리
----

In [76]:
# 최대 길이 찾기 for padding
max_input_len = 0
input_tokens_len = [len(x) for x in session_tokens]
max_input_len = max(max_input_len, max(input_tokens_len))    
    
batch_input_tokens = []
batch_input_attentions = []
for session_token in session_tokens:
    input_token = session_token + [fine_model.tokenizer.pad_token_id for _ in range(max_input_len-len(session_token))]
    input_attention = [1 for _ in range(len(session_token))] + [0 for _ in range(max_input_len-len(session_token))]
    batch_input_tokens.append(input_token)
    batch_input_attentions.append(input_attention)
    
batch_input_tokens = torch.tensor(batch_input_tokens).cuda()
batch_input_attentions = torch.tensor(batch_input_attentions).cuda()

In [77]:
softmax = torch.nn.Softmax(dim=1)
results = fine_model(batch_input_tokens, batch_input_attentions)
prob = softmax(results)
true_prob = prob[:,1].tolist()

print(context)
for utt, prob in zip(candidates, true_prob):
    print(utt, '##', round(prob,3))

['어떤 문제가 있으신가요?', '어떤 차를 사야 할지 잘 모르겠어요.', '차는 한 번 사면 10년도 넘게 써서, 신중하게 골라야 해요.']
저 좀 도와주실 수 있나요? ## 0.009
저는 농구를 좋아합니다. ## 0.002
자동차는 신중히 골라야합니다. ## 0.956
저는 차 마시는거를 좋아합니다. ## 0.134
날씨가 화창합니다. ## 0.004
저는 차를 좋아합니다. ## 0.947
